#  Neural Networks: Regression on House Pricing Dataset
We consider a reduced version of a dataset containing house sale prices for King County, which includes Seattle. It includes homes sold between May 2014 and May 2015.

https://www.kaggle.com/harlfoxem/housesalesprediction

For each house we know 18 house features (e.g., number of bedrooms, number of bathrooms, etc.) plus its price, that is what we would like to predict.

In [9]:
#put here your ``numero di matricola''
numero_di_matricola = 1

## Insert your ID number ("numero di matricola") below

In [10]:
#put here your ``numero di matricola''
numero_di_matricola = 1

In [11]:
#import all packages needed
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Load the data, remove data samples/points with missing values (NaN) and take a look at them.

In [12]:
#load the data
df = pd.read_csv('kc_house_data.csv', sep = ',')

#remove the data samples with missing values (NaN)
df = df.dropna() 

df.describe()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,3.164000e+03,3.164000e+03,3164.000000,3164.000000,3164.000000,3.164000e+03,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000
mean,4.645240e+09,5.354358e+05,3.381163,2.071903,2070.027813,1.525054e+04,1.434893,0.009798,0.244311,3.459229,7.615676,1761.252212,308.775601,1967.489254,94.668774,98077.125158,47.557868,-122.212337,1982.544564,13176.302465
std,2.854203e+09,3.809004e+05,0.895472,0.768212,920.251879,4.254457e+04,0.507792,0.098513,0.776298,0.682592,1.166324,815.934864,458.977904,28.095275,424.439427,54.172937,0.140789,0.139577,686.256670,25413.180755
min,1.000102e+06,7.500000e+04,0.000000,0.000000,380.000000,6.490000e+02,1.000000,0.000000,0.000000,1.000000,3.000000,380.000000,0.000000,1900.000000,0.000000,98001.000000,47.177500,-122.514000,620.000000,660.000000
25%,2.199775e+09,3.150000e+05,3.000000,1.500000,1430.000000,5.453750e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1190.000000,0.000000,1950.000000,0.000000,98032.000000,47.459575,-122.324250,1480.000000,5429.500000
50%,4.027701e+09,4.450000e+05,3.000000,2.000000,1910.000000,8.000000e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1545.000000,0.000000,1969.000000,0.000000,98059.000000,47.572500,-122.226000,1830.000000,7873.000000
75%,7.358175e+09,6.402500e+05,4.000000,2.500000,2500.000000,1.122250e+04,2.000000,0.000000,0.000000,4.000000,8.000000,2150.000000,600.000000,1990.000000,0.000000,98117.000000,47.680250,-122.124000,2360.000000,10408.250000
max,9.839301e+09,5.350000e+06,8.000000,6.000000,8010.000000,1.651359e+06,3.500000,1.000000,4.000000,5.000000,12.000000,6720.000000,2620.000000,2015.000000,2015.000000,98199.000000,47.777600,-121.315000,5790.000000,425581.000000


Extract input and output data. We want to predict the price by using features other than id as input.

In [13]:
Data = df.values
# m = number of input samples
m = Data.shape[0]
print("Amount of data:",m)
Y = Data[:m,2]
X = Data[:m,3:]

Amount of data: 3164


## Data Pre-Processing

We split the data into 3 parts: one will be used for training and choosing the parameters, one for choosing among different models, and one for testing. The part for training and choosing the parameters will consist of $2/3$ of all samples, the one for choosing among different models will consist of $1/6$ of all samples, while the other part consists of the remaining $1/6$-th of all samples.

In [14]:
# Split data into train (2/3 of samples), validation (1/6 of samples), and test data (the rest)
m_train = int(2./3.*m)
m_val = int((m-m_train)/2.)
m_test = m - m_train - m_val
print("Amount of data for training and deciding parameters:",m_train)
print("Amount of data for validation (choosing among different models):",m_val)
print("Amount of data for test:",m_test)
from sklearn.model_selection import train_test_split

Xtrain_and_val, Xtest, Ytrain_and_val, Ytest = train_test_split(X, Y, test_size=m_test/m, random_state=numero_di_matricola)
Xtrain, Xval, Ytrain, Yval = train_test_split(Xtrain_and_val, Ytrain_and_val, test_size=m_val/(m_train+m_val), random_state=numero_di_matricola)

Amount of data for training and deciding parameters: 2109
Amount of data for validation (choosing among different models): 527
Amount of data for test: 528


Let's standardize the data.

In [15]:
# Data pre-processing
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(Xtrain)
Xtrain_scaled = scaler.transform(Xtrain)
Xtrain_and_val_scaled = scaler.transform(Xtrain_and_val)
Xval_scaled = scaler.transform(Xval)
Xtest_scaled = scaler.transform(Xtest)

## Neural Networks
Let's start by learning a simple neural network with 1 hidden node.
Note: we are going to use the input parameter solver='lbfgs' and random_state=numero_di_matricola to fix the random seed (so results are reproducible).

In [16]:
#let's load the MLPRegressor

from sklearn.neural_network import MLPRegressor

#let's define the model
mlp = MLPRegressor(hidden_layer_sizes=(1, ), solver='lbfgs', random_state=numero_di_matricola)

#let's learn the model on training data
mlp.fit(Xtrain_scaled, Ytrain)

#let's print the error (1 - R^2) on training data
print("1 - coeff. det. on training data: ",1. - mlp.score(Xtrain_scaled, Ytrain))

#let's print the error (1 - R^2) on validation data
print("1 - coeff. det. on validation data: ",1. - mlp.score(Xval_scaled, Yval))

#let's print the coefficients of the model for the input nodes (but not the bias)
print(mlp.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
print(mlp.intercepts_)

1 - coeff. det. on training data:  0.26394803873968
1 - coeff. det. on validation data:  0.3040470894708266
[array([[-213.71151518],
       [ 268.29258679],
       [ 521.92389188],
       [ -60.46495569],
       [   3.9688407 ],
       [ 708.62929872],
       [ 293.35947487],
       [ 135.94386522],
       [ 813.57199353],
       [ 491.44655547],
       [ 163.02043652],
       [-580.18916615],
       [  37.79852046],
       [-202.78978492],
       [ 598.13959737],
       [-141.57154747],
       [ 146.50998396],
       [ -26.78838186]]), array([[141.60259856]])]
[array([3781.6473049]), array([-34.48971595])]


## Neural Networks vs Linear Models

Let's learn a linear model on the other same data and compare the results with the simple NN above.

In [17]:
from sklearn import linear_model

LR = linear_model.LinearRegression()

LR.fit(Xtrain_scaled, Ytrain)

#let's print the error (1 - R^2) on training data
print("1 - coeff. det. on training data: ",1. - LR.score(Xtrain_scaled, Ytrain))

#let's print the error (1 - R^2) on validation data
print("1 - coeff. det. on validation data: ",1. - LR.score(Xval_scaled, Yval))

print(LR.coef_)
print(LR.intercept_)

1 - coeff. det. on training data:  0.2653594216072852
1 - coeff. det. on validation data:  0.3115400506517969
[-31303.71909156  35848.45081517  74506.78099995  -8012.41104949
    671.23713588 100205.53195594  41671.19028923  19507.84532115
 111331.50566184  69959.22677526  23468.73219785 -78236.93092911
   6535.34729956 -28197.21476235  83701.76486765 -21647.26671149
  22056.22833416  -2002.69401407]
536831.9203413766


Is there a way to make a NN network learn a linear model?

Let's first check what is the loss used by MLPRegressor...

In [18]:
#let's write the code to learn a linear model with NN: how? 

mlp_lr = MLPRegressor(hidden_layer_sizes=(1, ), solver='lbfgs', random_state=numero_di_matricola, activation='identity')

#let's learn the model on training data
mlp_lr.fit(Xtrain_scaled, Ytrain)

#let's print the error (1 - R^2) on training data
print("1 - coefficient of determination on training data:"+str(1 - mlp_lr.score(Xtrain_scaled,Ytrain)))

#let's print the error (1 - R^2) on validation data
print("1 - coefficient of determination on validation data:"+str(1 - mlp_lr.score(Xval_scaled,Yval)))

#let's print the coefficients of the model for the input nodes (but not the bias)
print(mlp_lr.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
print(mlp_lr.intercepts_)

1 - coefficient of determination on training data:0.26535942166590454
1 - coefficient of determination on validation data:0.3115390658284921
[array([[  51.55070235],
       [ -59.02846704],
       [-122.96939596],
       [  13.19466306],
       [  -1.10694448],
       [-165.016906  ],
       [ -68.62491747],
       [ -32.12596414],
       [-183.34020234],
       [-114.96558392],
       [ -38.51551396],
       [ 128.83877577],
       [ -10.76237595],
       [  46.43749972],
       [-137.83874569],
       [  35.64844107],
       [ -36.32214078],
       [   3.29740834]]), array([[-607.24280969]])]
[array([-883.44724722]), array([365.29153421])]


Note that there is an $\ell_2$ regularization term in MLPRegressor. What about making it smaller?

In [19]:
#let's write the code to learn a linear model with NN: how? 

mlp_lr = MLPRegressor(hidden_layer_sizes=(1, ), solver='lbfgs', random_state=numero_di_matricola, activation='identity', alpha=1e-20)

#let's learn the model on training data
mlp_lr.fit(Xtrain_scaled, Ytrain)

#let's print the error (1 - R^2) on training data
print("1 - coefficient of determination on training data:"+str(1 - mlp_lr.score(Xtrain_scaled,Ytrain)))

#let's print the error (1 - R^2) on validation data
print("1 - coefficient of determination on validation data:"+str(1 - mlp_lr.score(Xval_scaled,Yval)))

#let's print the coefficients of the model for the input nodes (but not the bias)
print(mlp_lr.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
print(mlp_lr.intercepts_)

1 - coefficient of determination on training data:0.26535942166590454
1 - coefficient of determination on validation data:0.31153906582851487
[array([[  51.55070235],
       [ -59.02846704],
       [-122.96939596],
       [  13.19466306],
       [  -1.10694448],
       [-165.016906  ],
       [ -68.62491747],
       [ -32.12596414],
       [-183.34020234],
       [-114.96558392],
       [ -38.51551396],
       [ 128.83877577],
       [ -10.76237595],
       [  46.43749972],
       [-137.83874569],
       [  35.64844107],
       [ -36.32214078],
       [   3.29740834]]), array([[-607.24280969]])]
[array([-883.44724722]), array([365.29153421])]


## More Complex NNs

Let's try more complex NN, for example increasing the number of nodes in the only hidden layer, or increasing the number of hidden layers.

Let's build a NN with 2 nodes in the only hidden layer

In [20]:
#let's build a NN with 2 nodes in the only hidden layer

#let's define the model
mlp_1h2n = MLPRegressor(hidden_layer_sizes=(2, ), solver='lbfgs', random_state=numero_di_matricola)

#let's learn the model on training data
mlp_1h2n.fit(Xtrain_scaled, Ytrain)

#let's print the error (1 - R^2) on training data
print("1 - coefficient of determination on training data:"+str(1 - mlp_1h2n.score(Xtrain_scaled,Ytrain)))

#let's print the error (1 - R^2) on validation data
print("1 - coefficient of determination on validation data:"+str(1 - mlp_1h2n.score(Xval_scaled,Yval)))

#let's print the coefficients of the model for the input nodes (but not the bias)
print(mlp_1h2n.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
print(mlp_1h2n.intercepts_)

1 - coefficient of determination on training data:0.1806211007109051
1 - coefficient of determination on validation data:0.2071976726162894
[array([[  91.00132254,  -33.31841604],
       [ 120.42076004,   39.34037667],
       [  85.68119225,   73.03969183],
       [-260.92908033,   28.26456072],
       [ -30.64916845,   17.9186379 ],
       [ 198.03926939,   25.58876529],
       [  34.88962824,   37.56556254],
       [  96.95166772,   25.68935947],
       [ 312.45244185,  132.87069989],
       [  84.94301509,   68.83222969],
       [  19.2321553 ,   23.36629278],
       [-217.39281118,  -81.20985396],
       [  -3.4901083 ,   20.12046597],
       [-301.25060111,  -26.36495992],
       [ 305.5761844 ,  144.59688961],
       [-463.83892202,  -16.38586903],
       [ 194.5700832 ,   52.89698802],
       [-251.1689958 ,  -11.13406806]]), array([[614.59696206],
       [548.84336511]])]
[array([-1049.67248587,   897.60017347]), array([725.96981626])]


Let's build a NN with 5 nodes in the only hidden layer

In [21]:
#let's build a NN with 5 nodes in the only hidden layer
#mlp_1h5n = MLPRegressor(hidden_layer_sizes=(5, ), solver='lbfgs', random_state=numero_di_matricola)
#after warning shows up
mlp_1h5n = MLPRegressor(hidden_layer_sizes=(5, ), solver='lbfgs', random_state=numero_di_matricola, max_iter=2000)

#let's learn the model on training data
mlp_1h5n.fit(Xtrain_scaled, Ytrain)

#let's print the error (1 - R^2) on training data
print("1 - coefficient of determination on training data:"+str(1 - mlp_1h5n.score(Xtrain_scaled,Ytrain)))

#let's print the error (1 - R^2) on validation data
print("1 - coefficient of determination on validation data:"+str(1 - mlp_1h5n.score(Xval_scaled,Yval)))

#let's print the coefficients of the model for the input nodes (but not the bias)
print(mlp_1h5n.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
print(mlp_1h5n.intercepts_)

1 - coefficient of determination on training data:0.1370642895375931
1 - coefficient of determination on validation data:0.2512315028269181
[array([[ -282.71729987,    57.5660036 ,    84.32006365,   183.27499336,
          -27.84337387],
       [  986.07991577,   646.85200219,   171.6183778 ,  -180.58410753,
         -144.47806748],
       [ -388.5478935 ,     5.57416991,   208.00330104,  -411.07426885,
          440.47655866],
       [  451.26291185,  -713.35380204,  -109.6845454 ,  -808.86762377,
           60.99042515],
       [  466.89852603,  -115.10406495,   425.59109054,   233.43682073,
         -204.46842303],
       [ -878.72482833,   330.2783004 ,   889.55486601,  -529.2550437 ,
         -249.60034156],
       [ -205.9744776 ,   311.24510165,  -241.98019743,  -118.24289202,
          185.20300103],
       [ -423.04302744,   578.786409  ,  -226.99533856,   525.08120725,
           29.36039071],
       [  -22.83518067,   922.61170289,   811.76179848,   233.43811716,
          4

Note that with a smaller number of iterations we had a larger error on training set but a smaller error on validation data -> "early stopping is a form of regularization"

Let's build a NN with 10 nodes in the only hidden layer

In [22]:
#let's build a NN with 10 nodes in the only hidden layer

#let's define the model
mlp_1h10n = MLPRegressor(hidden_layer_sizes=(10, ), solver='lbfgs', random_state=numero_di_matricola)
#after warning shows up
#mlp_1h10n = MLPRegressor(hidden_layer_sizes=(10, ), solver='lbfgs', random_state=numero_di_matricola, max_iter=2000)

#let's learn the model on training data
mlp_1h10n.fit(Xtrain_scaled, Ytrain)

#let's print the error (1 - R^2) on training data
print("1 - coefficient of determination on training data:"+str(1 - mlp_1h10n.score(Xtrain_scaled,Ytrain)))

#let's print the error (1 - R^2) on validation data
print("1 - coefficient of determination on validation data:"+str(1 - mlp_1h10n.score(Xval_scaled,Yval)))

#let's print the coefficients of the model for the input nodes (but not the bias)
print(mlp_1h10n.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
print(mlp_1h10n.intercepts_)

1 - coefficient of determination on training data:0.12216726450701221
1 - coefficient of determination on validation data:0.3070216417103966
[array([[ 1.18144125e+02, -2.25944859e+01, -7.79119583e+01,
         7.24027789e+01,  5.51577464e+01, -5.27955860e+01,
         4.32928772e+01, -6.70820295e+00, -5.62185266e+01,
         1.87378496e+02],
       [-1.71412320e+01, -6.76373675e+01,  1.21121247e+02,
         5.11562110e+01, -8.10189835e+01,  4.15403466e+02,
         3.11927551e+02, -2.09067038e+01,  1.18832655e+02,
        -1.83543626e+02],
       [-7.17848349e+01,  2.68640367e+01,  2.48230966e+02,
         4.35717032e+01, -1.94713211e+02,  1.81221983e+02,
        -1.97859666e+02, -2.49629472e+00,  9.48228856e+01,
        -1.67681071e+02],
       [ 1.84002357e+02,  6.08346415e+01,  7.59803714e+01,
        -2.29878524e+02, -8.65333140e+01, -1.73350475e+01,
        -2.04741999e+02, -1.58305087e+02, -4.44697646e+01,
        -5.05650960e+02],
       [-6.10898180e+02,  5.75633448e+02, -1.8

/home/fogliodicarta/miniconda3/envs/ml2023/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Let's build a NN with 100 nodes in the only hidden layer. Note that this is the default!

In [23]:
#let's build a NN with 100 nodes in the only hidden layer

#let's define the model
#mlp_1h100n = MLPRegressor( solver='lbfgs', random_state=numero_di_matricola)
#after warning shows up
mlp_1h100n = MLPRegressor( solver='lbfgs', random_state=numero_di_matricola, max_iter=2000)

#let's learn the model on training data
mlp_1h100n.fit(Xtrain_scaled, Ytrain)

#let's print the error (1 - R^2) on training data
print("1 - coefficient of determination on training data:"+str(1 - mlp_1h100n.score(Xtrain_scaled,Ytrain)))

#let's print the error (1 - R^2) on validation data
print("1 - coefficient of determination on validation data:"+str(1 - mlp_1h100n.score(Xval_scaled,Yval)))

#let's print the coefficients of the model for the input nodes (but not the bias)
print(mlp_1h100n.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
print(mlp_1h100n.intercepts_)

1 - coefficient of determination on training data:0.00534128841412751
1 - coefficient of determination on validation data:0.8896166919469205
[array([[ 168.26497155,  -54.89412381,   44.85092641, ..., -173.38806126,
         209.18051806, -246.07889655],
       [  12.16782949,  -20.80449094,   75.79568137, ...,    6.43525356,
         -28.02556511,   62.96711115],
       [ -31.03611106, -114.26132764,   95.76456758, ...,   62.0297211 ,
         246.28672015,  -14.103095  ],
       ...,
       [ 265.37234095, -206.39826751,   92.94197596, ..., -403.30328866,
          67.26742488,  -95.34989884],
       [-123.70831184,  219.61659807,  175.17446827, ..., -151.89471898,
         -90.26041417,  -71.69066275],
       [  31.19581569,  -79.95971421,   72.17591778, ...,   60.54373967,
         116.73885679,   82.10595282]]), array([[ 436.34407628],
       [ 351.86911164],
       [-127.52866043],
       [-200.20401405],
       [ 391.50731282],
       [ 274.45059204],
       [ 374.51183257],
    

/home/fogliodicarta/miniconda3/envs/ml2023/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Let's try 2 layers, 1 node each

In [24]:
#let's build a NN with 2 hidden layers, 1 node each

#let's define the model
mlp_2h1n1n = MLPRegressor(hidden_layer_sizes=(1,1 ), solver='lbfgs', random_state=numero_di_matricola)
#after warning shows up
#mlp_2h2n2n = MLPRegressor(hidden_layer_sizes=(1,1 ), solver='lbfgs', random_state=numero_di_matricola, max_iter=2000)

#let's learn the model on training data
mlp_2h1n1n.fit(Xtrain_scaled, Ytrain)

#let's print the error (1 - R^2) on training data
print("1 - coefficient of determination on training data:"+str(1 - mlp_2h1n1n.score(Xtrain_scaled,Ytrain)))

#let's print the error (1 - R^2) on validation data
print("1 - coefficient of determination on validation data:"+str(1 - mlp_2h1n1n.score(Xval_scaled,Yval)))

#let's print the coefficients of the model for the input nodes (but not the bias)
print(mlp_2h1n1n.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
print(mlp_2h1n1n.intercepts_)

1 - coefficient of determination on training data:0.23857075595158783
1 - coefficient of determination on validation data:0.27110516474608704
[array([[ -30.13178629],
       [  85.93166509],
       [ 149.45416207],
       [ -42.61802713],
       [ -12.78198084],
       [ 174.12065813],
       [  69.29691506],
       [  42.17453568],
       [ 272.67821652],
       [ 149.18214618],
       [  32.13582509],
       [-186.62160069],
       [  12.21389767],
       [-133.83938695],
       [ 259.68919179],
       [-179.06728739],
       [  82.07321623],
       [ -10.55552054]]), array([[1.32886655]]), array([[434.1458692]])]
[array([140.48099925]), array([769.56890261]), array([498.79454679])]


/home/fogliodicarta/miniconda3/envs/ml2023/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Let's try 2 layers, 2 nodes each

In [25]:
#let's build a NN with 2 layers, 2 nodes each

#let's define the model
#mlp_2h2n2n = MLPRegressor(hidden_layer_sizes=(2,2 ), solver='lbfgs', random_state=numero_di_matricola)
#after warning shows up
mlp_2h2n2n = MLPRegressor(hidden_layer_sizes=(2,2 ), solver='lbfgs', random_state=numero_di_matricola, max_iter=2000)

#let's learn the model on training data
mlp_2h2n2n.fit(Xtrain_scaled, Ytrain)

#let's print the error (1 - R^2) on training data
print("1 - coefficient of determination on training data:"+str(1 - mlp_2h2n2n.score(Xtrain_scaled,Ytrain)))

#let's print the error (1 - R^2) on validation data
print("1 - coefficient of determination on validation data:"+str(1 - mlp_2h2n2n.score(Xval_scaled,Yval)))

#let's print the coefficients of the model for the input nodes (but not the bias)
print(mlp_2h2n2n.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
print(mlp_2h2n2n.intercepts_)

1 - coefficient of determination on training data:0.18976088361805876
1 - coefficient of determination on validation data:0.22445179393159198
[array([[-2.09847647e-01, -1.00378950e+02],
       [ 2.76565516e+00, -2.33486511e+03],
       [ 1.24024123e+00, -1.60201752e+02],
       [-2.34353052e+00,  2.19417176e+03],
       [-5.29191722e-01,  4.99166428e+02],
       [ 2.56784045e+00, -1.79908016e+03],
       [ 9.10299258e-01, -3.07582263e+02],
       [ 1.07099692e+00, -7.10232561e+02],
       [ 6.24547191e+00, -4.27088540e+03],
       [ 1.55151072e+00, -3.96757745e+02],
       [ 8.40950202e-02,  3.97463944e+02],
       [-5.45313524e+00,  4.31721166e+03],
       [-4.96706933e-01,  7.86887624e+02],
       [-3.99578176e+00,  3.10147521e+03],
       [ 4.35815829e+00, -2.24686801e+03],
       [-3.74304025e+00,  2.89611987e+03],
       [ 1.54900445e+00, -7.88138714e+02],
       [-1.28754704e+00,  9.50207277e+02]]), array([[-1.21884347e+02,  5.07830577e+01],
       [-9.45555318e+02,  6.12357052e-

Let's try 2 layers, 10 nodes each

In [26]:
#let's build a NN with 2 layers, 10 nodes each

#let's define the model
mlp_2h10n10n = MLPRegressor(hidden_layer_sizes=(10,10 ), solver='lbfgs', random_state=numero_di_matricola)
#after warning shows up
#mlp_2h10n10n = MLPRegressor(hidden_layer_sizes=(10,10 ), solver='lbfgs', random_state=numero_di_matricola, max_iter=2000)

#let's learn the model on training data
mlp_2h10n10n.fit(Xtrain_scaled, Ytrain)

#let's print the error (1 - R^2) on training data
print("1 - coefficient of determination on training data:"+str(1 - mlp_2h10n10n.score(Xtrain_scaled,Ytrain)))

#let's print the error (1 - R^2) on validation data
print("1 - coefficient of determination on validation data:"+str(1 - mlp_2h10n10n.score(Xval_scaled,Yval)))

#let's print the coefficients of the model for the input nodes (but not the bias)
print(mlp_2h10n10n.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
print(mlp_2h10n10n.intercepts_)

1 - coefficient of determination on training data:0.07783524241411877
1 - coefficient of determination on validation data:0.2842869114903399
[array([[  13.39322357,    8.33883203,   13.91647197,    5.42022778,
          -4.2604016 ,  -25.63640261,   -9.37955558,   -8.57324483,
           3.98375447,    6.30535703],
       [  -5.52519038,   26.32975463,   25.76973072,    1.16730701,
          -4.06381035,   41.79384218,  -16.90857181,   13.89429059,
          -6.08126622,   -1.85614484],
       [ -25.28815562,   15.41962243,    2.9098586 ,  -16.73318952,
         -12.21192566,    8.12593474,    3.69803088,   37.19940258,
         -38.05934198,   -6.06627626],
       [   1.08776033,    1.77537583,   12.06585769,  -16.9001802 ,
         -20.18584447,   -2.95622361,   -6.75175173,   -3.19738418,
           7.62893269,    1.67007446],
       [ -37.75097086,   36.04204523,  -22.18363216,    4.50998488,
          24.45874512,  -54.00161389,   25.18165311,  -39.00831012,
          25.80317621,

/home/fogliodicarta/miniconda3/envs/ml2023/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Let's try 2 layers, 100 nodes each

In [27]:
#let's build a NN with 2 layers, 100 nodes each

#let's define the model
#mlp_2h100n100n = MLPRegressor(hidden_layer_sizes=(100,100 ), solver='lbfgs', random_state=numero_di_matricola)
#after warning shows up
mlp_2h100n100n = MLPRegressor(hidden_layer_sizes=(100,100 ), solver='lbfgs', random_state=numero_di_matricola, max_iter=2000)

#let's learn the model on training data
mlp_2h100n100n.fit(Xtrain_scaled, Ytrain)

#let's print the error (1 - R^2) on training data
print("1 - coefficient of determination on training data:"+str(1 - mlp_2h100n100n.score(Xtrain_scaled,Ytrain)))

#let's print the error (1 - R^2) on validation data
print("1 - coefficient of determination on validation data:"+str(1 - mlp_2h100n100n.score(Xval_scaled,Yval)))

#let's print the coefficients of the model for the input nodes (but not the bias)
print(mlp_2h100n100n.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
print(mlp_2h100n100n.intercepts_)

1 - coefficient of determination on training data:0.001385328153442833
1 - coefficient of determination on validation data:0.5322336016417603
[array([[ 25.46382262,   1.44926751,   3.38232882, ...,  -4.59007989,
        -18.77545389,   3.83829845],
       [-15.45963534,   1.7029896 ,  21.48349796, ...,  -2.54616891,
         -2.1750276 , -21.61266617],
       [ -0.1064736 ,  -0.44550592,  13.6209323 , ...,   3.12240855,
         -8.30511166,  -5.87547765],
       ...,
       [  1.13610761,  -2.97835817,   8.95595784, ...,  31.20199747,
        -15.62244881,   6.3792036 ],
       [ -0.41614996,  -1.44274683, -17.52755238, ...,   2.1845111 ,
         42.21132892,   4.41196806],
       [-22.01940191,  -3.26303319,  16.42510358, ...,  -7.90121982,
         -4.45749477,   5.15747832]]), array([[ -2.43135087,  -5.82547061,  -1.03971283, ..., -12.98668613,
          1.75841123,  -4.21290416],
       [ -2.18654254,   0.1794134 ,  -0.02471706, ...,   0.12022166,
          0.34499111,   0.443225

/home/fogliodicarta/miniconda3/envs/ml2023/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


So it seems that 1 layer (and default number of iterations) works best for this dataset. Let's try 5-fold cross-validation with number of nodes in the hidden layer between 1 and 20.
Note that we use train and validation data together, since we are doing cross-validation.

In [28]:
from sklearn.model_selection import GridSearchCV

mlp_cv = MLPRegressor()
param_grid = {'hidden_layer_sizes': [i for i in range(1,21)],
              'activation': ['relu'],
              'solver': ['lbfgs'], 
              'random_state': [numero_di_matricola]
             }
mlp_GS = GridSearchCV(mlp_cv, param_grid=param_grid, 
                   cv=5, verbose=True)
mlp_GS.fit(Xtrain_and_val_scaled, Ytrain_and_val)

/home/fogliodicarta/miniconda3/envs/ml2023/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Fitting 5 folds for each of 20 candidates, totalling 100 fits


/home/fogliodicarta/miniconda3/envs/ml2023/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/home/fogliodicarta/miniconda3/envs/ml2023/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/home/fogliodicarta/miniconda3/envs/ml2023/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.

GridSearchCV(cv=5, estimator=MLPRegressor(),
             param_grid={'activation': ['relu'],
                         'hidden_layer_sizes': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10,
                                                11, 12, 13, 14, 15, 16, 17, 18,
                                                19, 20],
                         'random_state': [1], 'solver': ['lbfgs']},
             verbose=True)

Now let's check what is the best parameter, and compare the best NNs with the linear model (learned on train and validation) on test data.

In [29]:
##let's print the best model according to grid search
print("Best model: ",mlp_GS.best_estimator_)

#let's print the error 1-R^2 for the best model
print("Error (1-R^2) of best model: ",1. - mlp_GS.best_score_)

Best model:  MLPRegressor(hidden_layer_sizes=5, random_state=1, solver='lbfgs')
Error (1-R^2) of best model:  0.21276605544767857


Let compare the error of the best NN on train and validation and on test data.

In [30]:
print("Error best model on train and validation: ",1. - mlp_GS.best_estimator_.score(Xtrain_and_val_scaled,Ytrain_and_val))
print("Error best model on test data: ",1. - mlp_GS.best_estimator_.score(Xtest_scaled,Ytest))

Error best model on train and validation:  0.15067567331625298
Error best model on test data:  0.23042610431765742


Now let's learn the linear model on train and validation, and get error (1-R^2) on train and validation and on test data.

In [31]:
#LR the linear regression model
LR = linear_model.LinearRegression()

#fit the model on training data
LR.fit(Xtrain_and_val_scaled, Ytrain_and_val)

print("1 - coefficient of determination on training data:"+str(1 - LR.score(Xtrain_and_val_scaled,Ytrain_and_val)))
print("1 - coefficient of determination on test data:"+str(1 - LR.score(Xtest_scaled,Ytest)))

1 - coefficient of determination on training data:0.2715568577139226
1 - coefficient of determination on test data:0.3373644878767468


Note: MLPRegressor has several other parameters!